In [ ]:
import random
from prettytable import PrettyTable
from fpdf import FPDF
import json

# Dummy inventory data with prices in INR and stock count
with open("inventory.json", "r") as json_file:
    inventory = json.load(json_file)

# Function to match keywords, gender, budget, and consider stock count
def match_keywords_to_inventory(preprocessed_data, inventory):
    # Dummy matching logic
    matched_items = [
        item
        for item in inventory
        if (
            any(keyword.lower() in item["category"].lower() for keyword in preprocessed_data["preferred_categories"])
            and ((item["gender"].lower() == preprocessed_data["gender"].lower()) or (item["gender"].lower() == "u"))
            and (item["price"] <= preprocessed_data["budget"])
            and (item["stock_count"] > 0)  # Consider items with a positive stock count
        )
    ]
    return matched_items

# Example usage with budget and preferred categories input by numbers
user_name = input("Name: ")
mobile_number = input("Mobile Number: ")
occasion = "Birthday"
age = 30
gender = input("Gender (M/F): ")
address = input("Enter your address: ")
print("Categories:")
print("1. Clothing")
print("2. Jewelry")
print("3. Home Decor")
selected_categories = input("Select preferred categories (comma-separated numbers): ").split(",")
preferred_categories = []

for category_num in selected_categories:
    if category_num == "1":
        preferred_categories.append("Clothing")
    elif category_num == "2":
        preferred_categories.append("Jewelry")
    elif category_num == "3":
        preferred_categories.append("Home Decor")

budget = int(input("Budget: "))

# Step 1: Preprocess user inputs
processed_data = {
    "user_name": f"{user_name} - {mobile_number}",
    "occasion": occasion,
    "age": age,
    "gender": gender,
    "preferred_categories": preferred_categories,
    "budget": budget,
}

# Step 2: Match keywords, gender, budget, and consider stock count
matched_items = match_keywords_to_inventory(processed_data, inventory)
random.shuffle(matched_items)

# Display randomized recommendations to the user within the budget
print("\n")
print("Thank you for choosing giftaSPAC")
print(f"Hello {user_name}, here are your personalized randomized recommendations within your budget:")

order_number = 1
try:
    with open("order_details.json", "r") as order_file:
        orders = [json.loads(line) for line in order_file if line.strip()]
        if orders:
            order_number = max(order["order_number"] for order in orders) + 1
except (FileNotFoundError, ValueError):
    pass  # If the file doesn't exist, start with order_number = 1

if matched_items:
    total_price = 0
    table = PrettyTable(["Item", "Price", "Category", "Gender", "Stock Count"])
    ordered_items = []
    for item in matched_items:
        if total_price + item["price"] <= processed_data["budget"] and item["stock_count"] > 0:
            table.add_row([item["item"], item["price"], item["category"], item["gender"], item["stock_count"]])
            total_price += item["price"]
            ordered_items.append(item)

    if total_price == 0:
        print("Cannot predict within budget. Please adjust your preferences or budget.")
    else:
        print(table)
        print("Total price as per products:", total_price)
        print("Profit: ", budget - total_price)

        # Step 3: Ask the user to approve the order
        user_approval = input("Do you approve the order? (Y/N): ").upper()

        if user_approval == "Y":
            for ordered_item in ordered_items:
                for inv_item in inventory:
                    if inv_item["item"] == ordered_item["item"]:
                        inv_item["stock_count"] -= 1

            # Save the updated inventory to inventory.json
            with open("inventory.json", "w") as json_file:
                json.dump(inventory, json_file, indent=2)

            # Step 4: Generate a bill as a PDF
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.image("https://abhinavchebrolu.co.in/SPAC/images/Logo%20red.png", x=10, y=8, w=33)

            pdf.set_font("Times", size=14)

            pdf.cell(200, 10, txt=f"Tax Invoice/Bill of Supply - Order Confirmation \t\t\t", ln=True, align="R")

            pdf.set_font("Arial", "B", size=12)
            pdf.cell(200, 5, txt=f"Order Number: SPAC000{order_number}\t\t\t", ln=True, align="R")
            pdf.ln(9)
            pdf.cell(200, 5, txt=f"CustName: {user_name}", ln=True)
            pdf.cell(200, 5, txt=f"Mobile: {mobile_number}", ln=True)
            pdf.cell(200, 5, txt=f"CustAddress: {address}", ln=True)
            pdf.cell(200, 5, txt=f"Occasion: Happy {occasion}", ln=True)
            pdf.ln(7)

            pdf.set_font("Arial", size=12)

            # Adding the table to the PDF
            pdf.cell(200, 5, txt="Here you go, SPAC's Recommended Products:", ln=True)
            pdf.ln(5)

            # Set column widths for the table
            col_widths = [80, 25, 35, 20, 30]

            # Set background color for the first row
            pdf.set_fill_color(192, 192, 192)  # RGB values for grey color

            # Add table headers with background color
            for i in range(len(table.field_names)):
                pdf.cell(col_widths[i], 10, txt=str(table.field_names[i]), border=1, align="C", fill=True)
            pdf.ln()

            # Reset background color for subsequent rows
            pdf.set_fill_color(255, 255, 255)  # Reset to white background

            # Add table rows
            for row in table._rows:
                for i in range(len(row)):
                    pdf.cell(col_widths[i], 10, txt=str(row[i]), border=1)
                pdf.ln()

            pdf.ln(10)
            pdf.cell(200, 5, txt=f"Total Price: {total_price}", ln=True)
            pdf.cell(200, 5, txt="Thank you for choosing giftaSPAC!", ln=True)

            pdf.output(f"order_confirmation_SPAC000{order_number}.pdf")
            order_details = {
                "order_number": order_number,
                "user_name": user_name,
                "mobile": mobile_number,
                "address": address,
                "gender": gender,
                "occasion": occasion,
                "items": [
                    {
                        "item": item["item"],
                        "price": item["price"],
                        "category": item["category"],
                        "gender": item["gender"],
                    }
                    for item in ordered_items
                ],
                "total_price": total_price,
            }

            with open("order_details.json", "a") as order_file:
                json.dump(order_details, order_file, indent=2)
                order_file.write("\n")
            print(f"Order confirmed! Order details have been saved in order_confirmation_SPAC000{order_number}.pdf and order_details.json.")
            order_number += 1
        else:
            print("Order canceled.")
else:
    print("Cannot predict within budget. Please adjust your preferences or budget.")
